# Example for Marianna

Here we simulate a dSPN and an iSPN neuron, and see what the effect is on clamping PKAc first to the half activation value 3.6e-6mM (baseline), then to 0 (for ispn), and to 50e-6mM (for dspn)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.io as pio 
pio.templates.default = "simple_white"

## Create network

First we create a small example network with dSPN and iSPN neurons. You can change the number of neurons by editing the ```number_of_neurons``` below. The density is important, as it affects the connectivity, since connectivity depends on distance (we are using touch detection to detect synapses and gap junctions).

You need to change the paths that use ```/home/hjorth/HBP``` to point to where you stored the information. ```BasalGangliaData``` can be found here ```https://github.com/hjorthmedh/BasalGangliaData/```

In [ ]:
import os
from snudda import Snudda


neuron_path_dspn = "/home/hjorth/HBP/BasalGangliaData/data/neurons/striatum/dspn/str-dspn-e150917_c10_D1-mWT-P270-20-v20220620"
neuron_path_ispn = "/home/hjorth/HBP/BasalGangliaData/data/neurons/striatum/ispn/str-ispn-e160118_c10_D2-m46-3-DE-v20220622"

snudda_data = "/home/hjorth/HBP/BasalGangliaData/data"

network_path = os.path.join("..", "networks", "neuromodulation_PKAc_synaptic_input")

sim_time = 2.0
sim_config_on = "da_experiment_synaptic_on_bath_sbml.json"
sim_config_off = "da_experiment_synaptic_off_bath_sbml.json"
mech_dir = "/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms"

sim_output_neuromodulation_ON = os.path.join(network_path, "simulation", "output_neuromodulation_ON.hdf5")
sim_output_neuromodulation_OFF = os.path.join(network_path, "simulation", "output_neuromodulation_OFF.hdf5")

In [ ]:
snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=[neuron_path_dspn, neuron_path_ispn], 
                      neuron_names=["dSPN", "iSPN"], number_of_neurons=[10, 10],
                      snudda_data=snudda_data,
                      density=80500,
                      d_min=15e-6,
                      random_seed=123)

si.network_data["regions"]["Cube"]["neurons"]["dSPN"]["reaction_diffusion"] = "reaction_diffusion_D1_empty.json"
si.network_data["regions"]["Cube"]["neurons"]["dSPN"]["modulation"] = "./modulation_parameters.json"
si.network_data["regions"]["Cube"]["neurons"]["dSPN"]["modulation_key"] = "abc"

si.network_data["regions"]["Cube"]["neurons"]["iSPN"]["reaction_diffusion"] = "reaction_diffusion_D2_empty.json"
si.network_data["regions"]["Cube"]["neurons"]["iSPN"]["modulation"] = "./modulation_parameters.json"
si.network_data["regions"]["Cube"]["neurons"]["iSPN"]["modulation_key"] = "abc"


si.network_data["regions"]["Cube"]["neurons"]["dSPN"]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

In [ ]:
snd = Snudda(network_path=network_path, parallel=False)
snd.create_network()

## Create synaptic input

The file ```marianna_input.json``` defines the input here.

In [ ]:
snd.setup_input(input_config="marianna_input.json")


## Run the simulation

The Jupyter notebook can be tricky, sometimes the os.system call does not work (if you run without the ```mpirun``` it will work, but then it is slower). You might have to copy paste the shell command to the terminal manually, just be sure you are in the same folder when you run it.

In [ ]:
number_of_cores = 8

In [ ]:
run_str_on = f"mpirun -n {number_of_cores} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on} --mechdir {mech_dir} --enable_rxd_neuromodulation --verbose"
print(run_str_on)

In [ ]:
os.system(run_str_on)

In [ ]:
run_str_off = f"mpirun -n {number_of_cores} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_off} --mechdir {mech_dir}"
print(run_str_off)

In [ ]:
os.system(run_str_off)

## Inspect the simulation

This simulation is run with neuromodulation, the first few figures below inspect the state of the neuromodulation.

In [ ]:
from snudda.utils import SnuddaLoadSimulation

nd = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
time = nd.get_time()

# Double check that 0 is dSPN and 1 is iSPN
data_pka_dspn = nd.get_data("PKAc", 0)[0][0]
data_pka_ispn = nd.get_data("PKAc", 1)[0][1]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

In [ ]:
# This is just so you know what neurons are where
nd.network_simulation_file["meta_data"]["name"][()]

### Plot internal state

This plot is quite messy, we have used it to debug the state. You can double click on the legends, to see the individual curves. It lets you inspect the level of PKAc (proxy for neuromodulation), the currents (e.g. bk_ms.ik), and the modulation (e.g. cal12_ms.modulation_factor).

In [ ]:
neuron_id = 1

data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=neuron_id, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [neuron_id])

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG


fig = go.Figure()
for data_type in all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][neuron_id].T[0][idx], name=data_type, line={"width":4}))

"""
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                 font={"size":18},  # General font size for all elements
                 legend={"font":{"size":16}},  # Specific font size for legend
                 xaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}},  # X-axis title and tick labels
                 yaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}})   # Y-axis title and tick labels
"""
fig.show()

In [ ]:
from snudda.plotting.plotly.trace import PlotTrace

pt_on = PlotTrace(snudda_load_simulation=nd)
# pt_on.define_colour_by_neuron_id({0: "blue", 1: "red"})
pt_on.define_colour_by_neuron_id({0: "blue", 1: "blue", 2: "blue", 3: "blue", 4: "blue", 5: "blue", 6: "blue", 7: "blue", 8: "blue", 9: "blue", 
                                  10: "red", 11: "red", 12: "red", 13: "red", 14: "red", 15: "red", 16: "red", 17: "red", 18: "red", 19: "red"})

pt_on.plot_traces()

In [ ]:
from snudda.utils import SnuddaLoadSimulation

nd_off = SnuddaLoadSimulation(sim_output_neuromodulation_OFF)
time = nd_off.get_time()


In [ ]:
# DA Off

from snudda.plotting.plotly.trace import PlotTrace

pt_off = PlotTrace(snudda_load_simulation=nd_off)
pt_off.define_colour_by_neuron_id({0: "blue", 1: "blue", 2: "blue", 3: "blue", 4: "blue", 5: "blue", 6: "blue", 7: "blue", 8: "blue", 9: "blue", 
                                   10: "red", 11: "red", 12: "red", 13: "red", 14: "red", 15: "red", 16: "red", 17: "red", 18: "red", 19: "red"})
pt_off.plot_traces()